## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

To Link ur custom artifact path used for tracking run,navigate to the path and run: 

 `mlflow server  --backend-store-uri  sqlite:///backend.db  --default-artifact-root  ./mlruns-local` 


In [9]:
import mlflow

# TRACKING_URI = "file:///workspaces/mlops-zoomcamp-2025/02-experiment-tracking/running-mlflow-examples/scenario-2/mlruns"

# mlflow.set_tracking_uri(TRACKING_URI)

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1745908299213, experiment_id='0', last_update_time=1745908299213, lifecycle_stage='active', name='Default', tags={}>]

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/04/29 06:42:31 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/04/29 06:42:34 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpgry54wab/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/04/29 06:42:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/workspaces/mlops-zoomcamp-2025/02-experiment-tracking/running-mlflow-examples/scenario-2/mlruns-local/1/579b1c41428d45cc92964f2069148d1c/artifacts'
🏃 View run delicate-cod-412 at: http://127.0.0.1:5000/#/experiments/1/runs/579b1c41428d45cc92964f2069148d1c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [13]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/02-experiment-tracking/running-mlflow-examples/scenario-2/mlruns-local/1', creation_time=1745908951493, experiment_id='1', last_update_time=1745908951493, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1745908299213, experiment_id='0', last_update_time=1745908299213, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [15]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [16]:
client.search_registered_models()

[]

In [17]:
# Ensure the experiment ID is valid and exists
experiment_id = mlflow.search_experiments()[0].experiment_id  # Use the first experiment as an example

# Search for runs in the experiment
runs = client.search_runs(experiment_ids=[experiment_id])

if runs:
    run_id = runs[0].info.run_id  # Get the run ID of the first run
    mlflow.register_model(
        model_uri=f"runs:/{run_id}/models",
        name='iris-classifier'
    )
else:
    print("No runs found for the specified experiment.")

Successfully registered model 'iris-classifier'.
2025/04/29 06:43:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.
